## Tests for Chebyshev interpolation with cross approximation (TT-format)

> Interpolation of functions by Chebyshev polynomials via cross approximation in the tensor train (TT) format.

In [1]:
import sys
import time

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import tt

sys.path.append('./../lib')
import cheb.cheb_tt as ch
import cheb.utils as chu
import cheb.cross as cr

ModuleNotFoundError: No module named 'cheb.cross'

### Function for approximation and interpolation:

$$ f(x) = 1 + e^{-0.5} x^2_1sin(c_1) + e^{-1} x^3_2 sin(2c_2) + \sum_{k=3}^{n} e^{-0.5k} x_k sin(k c_k); \;\; c=const, \; n>2 $$

In [ ]:
def func(x, c):
    k = np.arange(len(x), dtype=float) + 1.
    l = np.exp(-0.5*k)
    f = l * x * np.sin(k * c)
    f[0] *= x[0]
    f[1] *= x[1] * x[1]
    return 1. + np.sum(f)

# Parameters of the calculation

In [ ]:
n_dim = 10
c = np.ones(n_dim)
n_poi_cr = np.array([10]*n_dim)
x_lim_cr = np.array([[-3., 3.]]*n_dim)
tol_cr = 1.E-7; tol_tt = 1.E-7
file_log = './tmp/cross_appr_log.txt'

### Cross approximation

In [ ]:
Time = time.time()
f = func; f_args = [c]
Y, calc_num, cr_info = cr_appr.calculate_tensor(f, f_args, n_poi_cr, 
                                                x_lim_cr, tol_cr, file_log)
cr_iter, cr_err_rel, cr_err_abs, cr_e_rank = cr_info
print "Number of function's calculations: ", calc_num
print "               * full tensor size: ", np.prod(np.array(n_poi_cr))
print "Ranks of Y tensor                : ", Y.r
print "Average tensor's ranks           : ", cr_e_rank
print "Number of cross iterations       : ", cr_iter
print "Relative error of approximation  : ", cr_err_rel
print "Absolute error of approximation  : ", cr_err_abs
G = tt.tensor.to_list(Y); 
Ysize = 0; 
for GG in G: Ysize += GG.size
print "Total size of Y cores            : ", Ysize
print '----------------------'
print 'Total time %6.3f sec. '%(time.time() - Time)

### Interpolation by Chebyshev polynomials

In [ ]:
Time = time.time()
A = ch_tt.interpolate(Y, tol_tt)
print "Ranks of A tensor    : ", A.r
G = tt.tensor.to_list(A)
Asize = 0
for GG in G: Asize += GG.size
print "Total size of A cores: ", Asize
print '----------------------'
print 'Total time %6.3f sec. '%(time.time() - Time)

### Check of interpolation result on the set of random points

In [ ]:
Time = time.time()
poi_num = 1000; a, b = -3., 3.
X = a + np.random.random((n_dim, poi_num))*(b - a)
f_real = [func(X[:, i], c) for i in range(poi_num)]
f_calc = ch_tt.func_val(X, A, x_lim_cr)
print 'Maximum relative error of results on check mesh: ', 
print '%-8.2e'%np.max(np.abs((f_calc-f_real) / f_real))
print 'Mean relative error of results on check mesh   : ', 
print '%-8.2e'%np.mean(np.abs((f_calc-f_real) / f_real))
print '----------------------'
print 'Total time %6.3f sec. '%(time.time() - Time)

In [ ]:
gs = gridspec.GridSpec(1, 2)
gs.update(left=0.05, right=1.5, wspace=0.4)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])
ax1.set_title('Real and approximated function values on test points')
ax2.set_title('Relative error')
ax1.plot(np.arange(poi_num)+1, f_real, label='Real') 
ax1.plot(np.arange(poi_num)+1, f_calc, label='Appr')
ax2.plot(np.arange(poi_num)+1, np.abs((f_real-f_calc)/f_real))
ax1.legend(loc='best'); ax2.semilogy(); plt.show()